In [1]:
# importing useful packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
plt.style.use('seaborn-white')

## Writing out WISE W1 and W2 magnitude errors

### Columns of 'wiseScosSvm':

'wiseX', 'wiseID', 'scosID', 'cx', 'cy', 'cz', 'htmID', 'ra', 'dec',
'ebv', 'w1mCorr', 'w1mCorrErr', 'w2mCorr', 'w2mCorrErr', 'bCalCorr',
'bCalCorrErr', 'rCalCorr', 'rCalCorrErr', 'zPhoto_ANN', 'zPhoto_Corr',
'fromAllSky', 'SVMClass', 'probGal', 'probStar', 'probQso',
'inPhotozCat'

**Reading the wiseScosSvm catalog (http://ssa.roe.ac.uk/www/ssa_browser.html):**

In [2]:
wxs = pd.read_csv( 'wiseScosSvm.csv', usecols = [ 'wiseX', 'w1mCorr', 'w1mCorrErr', 
                                                 'w2mCorr', 'w2mCorrErr', 'fromAllSky' ], 
                    header = 0, low_memory = False ) 
wxs.head()

,wiseX,w1mCorr,w1mCorrErr,w2mCorr,w2mCorrErr,fromAllSky
0,J113821.86-492730.2,15.578,0.044,15.482,0.081,0
1,J113829.99-492503.1,15.403,0.037,15.483,0.091,0
2,J113821.16-492311.9,15.139,0.036,15.176,0.066,0
3,J113817.76-491634.5,15.690,0.039,15.560,0.082,0
4,J113823.10-492513.8,15.381,0.038,15.332,0.083,0


**Looking for unique ID-s:**

In [3]:
u, c = np.unique( wxs.wiseX.values.astype(str), return_counts=True )

The number of identical ID-s:

In [4]:
(c > 1).sum()

24675

**Converting the ID-s to integers to speed up the code:**

In [5]:
id_to_int = dict( zip( wxs.wiseX.values.astype(str), np.arange(wxs.shape[0]) ) )

fromAllSky: Is the WISE designation from the wise_allskysc catalogue 0 NO/1 YES

In [6]:
wxs_0 = wxs[ wxs.fromAllSky.values == 0 ]
wxs_1 = wxs[ wxs.fromAllSky.values == 1 ]

**Giving the integer ID-s:**

In [7]:
wxs_0['id'] = [ id_to_int[i] for i in wxs_0.wiseX.values ]

/home/marika/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
wxs_1['id'] = [ id_to_int[i] for i in wxs_1.wiseX.values ]

/home/marika/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Lengths:

In [9]:
wxs_0.shape[0], wxs_1.shape[0]

(38374666, 1441454)

**If we cannot find the galaxy in the AllWISE catalog, we use the data from the AllSky catalog.**

In [10]:
wxs_new = pd.DataFrame( np.concatenate( ( wxs_0.values[:,:-1], wxs_1[ ~np.in1d( wxs_1.id.values, wxs_0.id.values) ].values[:,:-1] ), axis=0 ), 
              columns=['wiseX', 'w1mCorr', 'w1mCorrErr', 'w2mCorr', 'w2mCorrErr', 'fromAllSky'])
wxs_new.head()

,wiseX,w1mCorr,w1mCorrErr,w2mCorr,w2mCorrErr,fromAllSky
0,J113821.86-492730.2,15.578,0.044,15.482,0.081,0
1,J113829.99-492503.1,15.403,0.037,15.483,0.091,0
2,J113821.16-492311.9,15.139,0.036,15.176,0.066,0
3,J113817.76-491634.5,15.69,0.039,15.56,0.082,0
4,J113823.10-492513.8,15.381,0.038,15.332,0.083,0


How many galaxies do we have?

In [11]:
len(wxs_new)

39791445

**Writing the results to a txt file:**

In [12]:
# write out data
f_out = 'wise_err.txt'
wxs_new.to_csv( f_out, sep = '\t', mode = 'w', header = True, index = False )